In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pandas as pd
from scipy import stats

from src.dataset import load_data
from src.features import build_features, compare_distributions

Code from here: https://github.com/shreyashankar/debugging-ml-talk/blob/main/nyc_taxi_2020.ipynb

## Loading data

In [2]:
taxi_jan = load_data(year=2020, month=1)
taxi_feb = load_data(year=2020, month=2)
taxi_mar = load_data(year=2020, month=3)

In [3]:
len(taxi_jan), len(taxi_feb), len(taxi_mar)

(6405008, 6299367, 3007687)

## Pre-processing

In [4]:
target_col = "high_tip"

taxi_jan = build_features(taxi_jan.sample(100000), target_col)
taxi_feb = build_features(taxi_feb.sample(100000), target_col)
taxi_mar = build_features(taxi_mar.sample(100000), target_col)

## Test distributions

Maybe the distribution of data shifted. We could try to quantify this using a 2-sided statistical test (Kolmogorov Smirnov in this example).

### Compare January 2020 vs February 2020

The p values being small doesn't really tell us much, as we get very small p values when comparing January 2020 vs February 2020 even though we know the F1 score was similar. Curse "big data."

In [5]:
comparison_df = compare_distributions(taxi_jan, taxi_feb)
comparison_df.sort_values(by='p_value', ascending=True).head(11)

,feature,statistic,p_value
0,pickup_weekday,0.044177,6.218576e-85
7,trip_speed,0.033741,1.027005e-49
2,work_hours,0.027220,1.670956e-32
6,trip_time,0.021149,8.635545e-20
1,pickup_hour,0.007307,9.729757e-03
5,trip_distance,0.006918,1.689752e-02
8,PULocationID,0.005181,1.373429e-01
9,DOLocationID,0.005150,1.417980e-01
10,RatecodeID,0.003821,4.597957e-01
4,passenger_count,0.002338,9.476950e-01


### Compare February 2020 vs March 2020

These p values are also small, which is good? But if this method in general sends warning alerts all the time, an end user might not trust it.

In [6]:
comparison_df = compare_distributions(taxi_feb, taxi_mar)
comparison_df.sort_values(by='p_value', ascending=True).head(11)

,feature,statistic,p_value
0,pickup_weekday,0.062646,2.172041e-170
7,trip_speed,0.062599,3.910151e-170
2,work_hours,0.039146,9.238106e-67
6,trip_time,0.024335,4.670996e-26
1,pickup_hour,0.024328,4.818571e-26
4,passenger_count,0.022211,8.852840e-22
5,trip_distance,0.019368,1.157111e-16
9,DOLocationID,0.016817,1.145906e-12
8,PULocationID,0.015528,7.299004e-11
10,RatecodeID,0.004651,2.306753e-01
